In [1]:
import cvxpy as cp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

(CVXPY) Dec 06 12:25:21 PM: Encountered unexpected exception importing solver OSQP:
ImportError('DLL load failed while importing qdldl: 找不到指定的模块。')


In [34]:
# Read the dataset from the file
data = pd.read_csv('car+evaluation/car.data', header=None)
data = pd.get_dummies(data)

# Display the first few rows of the dataset
data.info()
data.iloc[:5,:]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 25 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   0_high   1728 non-null   bool 
 1   0_low    1728 non-null   bool 
 2   0_med    1728 non-null   bool 
 3   0_vhigh  1728 non-null   bool 
 4   1_high   1728 non-null   bool 
 5   1_low    1728 non-null   bool 
 6   1_med    1728 non-null   bool 
 7   1_vhigh  1728 non-null   bool 
 8   2_2      1728 non-null   bool 
 9   2_3      1728 non-null   bool 
 10  2_4      1728 non-null   bool 
 11  2_5more  1728 non-null   bool 
 12  3_2      1728 non-null   bool 
 13  3_4      1728 non-null   bool 
 14  3_more   1728 non-null   bool 
 15  4_big    1728 non-null   bool 
 16  4_med    1728 non-null   bool 
 17  4_small  1728 non-null   bool 
 18  5_high   1728 non-null   bool 
 19  5_low    1728 non-null   bool 
 20  5_med    1728 non-null   bool 
 21  6_acc    1728 non-null   bool 
 22  6_good   1728 non-null  

,0_high,0_low,0_med,0_vhigh,1_high,1_low,1_med,1_vhigh,2_2,2_3,...,4_big,4_med,4_small,5_high,5_low,5_med,6_acc,6_good,6_unacc,6_vgood
0,False,False,False,True,False,False,False,True,True,False,...,False,False,True,False,True,False,False,False,True,False
1,False,False,False,True,False,False,False,True,True,False,...,False,False,True,False,False,True,False,False,True,False
2,False,False,False,True,False,False,False,True,True,False,...,False,False,True,True,False,False,False,False,True,False
3,False,False,False,True,False,False,False,True,True,False,...,False,True,False,False,True,False,False,False,True,False
4,False,False,False,True,False,False,False,True,True,False,...,False,True,False,False,False,True,False,False,True,False


In [3]:
#convert all columns to numeric
for i in range(0, 6):
    data[i] = pd.Categorical(data[i])
    data[i] = data[i].cat.codes

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

columns = [0,1,2,3,4,5]

train_X, test_X, train_y, test_y = train_test_split(data[columns], data[6].replace({'unacc': -1, 'acc': 1, 'good': 1, 'vgood': 1}), test_size=0.3)

# Create an instance of the StandardScaler
scaler = MinMaxScaler()

# Normalize the data
train_X = scaler.fit_transform(train_X)
test_X = scaler.transform(test_X)

train_X

array([[0.33333333, 1.        , 0.33333333, 0.5       , 1.        ,
        0.        ],
       [1.        , 0.33333333, 0.33333333, 0.5       , 0.        ,
        1.        ],
       [0.66666667, 0.        , 1.        , 1.        , 0.        ,
        1.        ],
       ...,
       [0.        , 0.        , 0.66666667, 0.        , 1.        ,
        0.        ],
       [0.66666667, 1.        , 0.66666667, 1.        , 1.        ,
        0.        ],
       [1.        , 0.        , 0.66666667, 1.        , 0.5       ,
        0.        ]])

In [5]:
def gaussian_kernel(x, y, sigma=1.0):
    return np.exp(-np.linalg.norm(x - y) ** 2 / (2 * (sigma ** 2)))
def linear_kernel(x, y):
    return np.dot(x, y)
kernel_matrix = np.array([[gaussian_kernel(x, y) for y in train_X] for x in train_X])
kernel_matrix


array([[1.        , 0.23587708, 0.14915373, ..., 0.47897297, 0.78969293,
        0.35780119],
       [0.23587708, 1.        , 0.63233666, ..., 0.17620431, 0.23262366,
        0.42269216],
       [0.14915373, 0.63233666, 1.        , ..., 0.16901332, 0.21107209,
        0.47897297],
       ...,
       [0.47897297, 0.17620431, 0.16901332, ..., 1.        , 0.29457483,
        0.32465247],
       [0.78969293, 0.23262366, 0.21107209, ..., 0.29457483, 1.        ,
        0.50633562],
       [0.35780119, 0.42269216, 0.47897297, ..., 0.32465247, 0.50633562,
        1.        ]])

In [6]:
alpha = cp.Variable((train_X.shape[0],1))
outer_y = train_y.values.reshape((-1,1)) * train_y.values.reshape((-1,1)).T
gram = outer_y * kernel_matrix
objective = cp.Minimize(-cp.sum(alpha) + 0.5 * cp.quad_form(alpha, cp.psd_wrap(gram)))
constraints = [0 <= alpha, alpha <= 1, train_y.values @ alpha == 0]

problem = cp.Problem(objective, constraints)
problem.solve(solver = cp.SCS)
print("problem status: ",problem.status)

problem status:  optimal


In [7]:
# support_vector_indices = np.where(alpha.value > 1e-5)[0]
# support_vectors = train_X[support_vector_indices]
# support_vector_labels = train_y.values[support_vector_indices]

test_kernel_matrix = np.array([[gaussian_kernel(x, y) for y in test_X] for x in train_X])
predictions = np.sign(np.sum(train_y.values.reshape((-1,1)) * alpha.value * test_kernel_matrix, axis=0))
print('pridiction_type',np.unique(predictions,return_counts=True))
accuracy = np.mean(predictions == test_y.values)

print(f"Accuracy: {accuracy}")

pridiction_type (array([-1.,  1.]), array([197, 322], dtype=int64))
Accuracy: 0.6994219653179191


In [8]:
gamma_list = [0, 0.001, 0.005, 0.01, 0.05, 0.1, 1]
train_X, test_X, train_y, test_y = train_test_split(data[columns], data[6], test_size=0.1)
train_X = scaler.fit_transform(train_X)
test_X = scaler.transform(test_X)

In [9]:

# Train the SVM classifier
def train_svm(X_train, y_train, X_test, y_test, gamma):
    m = X_train.shape[0]
    n = X_train.shape[1]
    y_train = y_train.reshape((-1,1))
    y_test = y_test.reshape((-1,1))
    # L1 norm
    w1 = cp.Variable((n,1))
    b1 = cp.Variable()
    n1 = cp.Variable((m,1))
    const = np.ones((m,1))
    constraints1 = [cp.multiply(y_train, (X_train @ w1 + b1)) >= const - n1, n1 >= 0]
    objective1 = cp.Minimize(cp.norm1(w1) + gamma * cp.norm1(n1))
    problem1 = cp.Problem(objective1, constraints1)
    problem1.solve(verbose=False)
    predictions1 = np.sign(X_test @ w1.value + b1.value)
    accuracy1 = np.mean(predictions1 == y_test)

    # L2 norm
    w2 = cp.Variable((n,1))
    b2 = cp.Variable()
    n2 = cp.Variable((m,1))
    constraints2 = [cp.multiply(y_train, (X_train @ w2 + b2)) >= const - n2, n2 >= 0]
    objective2 = cp.Minimize(cp.norm2(w2) + gamma * cp.norm1(n2))
    problem2 = cp.Problem(objective2, constraints2)
    problem2.solve(verbose=False)
    predictions2 = np.sign(X_test @ w2.value + b2.value)
    accuracy2 = np.mean(predictions2 == y_test)


    best_w = w1.value if accuracy1 > accuracy2 else w2.value
    best_b = b1.value if accuracy1 > accuracy2 else b2.value
    best_accuracy = accuracy1 if accuracy1 > accuracy2 else accuracy2
    return best_w, best_b, best_accuracy

In [10]:
def get_best_model(X_train, y_train, X_test, y_test):
    best_accuracy = 0
    best_w = None
    best_b = None
    for gamma in gamma_list:
        w, b, accuracy = train_svm(X_train, y_train, X_test, y_test, gamma)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_w = w
            best_b = b
    return best_w, best_b, best_accuracy


In [11]:
# Try different values of gamma
def train_general():
    # Train a SVM classifier, suppose acceptable is -1 and unacceptable, good, very good are 1
    train_y1 = train_y.replace({'unacc': -1, 'acc': 1, 'good': 1, 'vgood': 1}).values
    test_y1 = test_y.replace({'unacc': -1, 'acc': 1, 'good': 1, 'vgood': 1}).values
    best_w1, best_b1, best_accuracy = get_best_model(train_X, train_y1, test_X, test_y1)
    print('train_gegeral')
    print('predict_y1_types', np.unique(np.sign(test_X @ best_w1 + best_b1), return_counts=True))
    print(f"Accuracy: {best_accuracy}")

def train_vote():
    print('train_vote')
    # Train a SVM classifier, suppose unacceptable is -1 and acceptable is 1
    train_y2 = train_y[~train_y.isin(['good', 'vgood'])].replace({'unacc': -1, 'acc': 1}).values
    test_y2 = test_y[~test_y.isin(['good', 'vgood'])].replace({'unacc': -1, 'acc': 1}).values
    train_X2 = train_X[~train_y.isin(['good', 'vgood'])]
    test_X2 = test_X[~test_y.isin(['good', 'vgood'])]
    w2, b2, _ = get_best_model(train_X2, train_y2, test_X2, test_y2)
    print('predict_y2_types', np.unique(np.sign(test_X2 @ w2 + b2), return_counts=True))
    
    # Train a SVM classifier, suppose unacceptable is -1 and good is 1
    train_y3 = train_y[~train_y.isin(['acc', 'vgood'])].replace({'unacc': -1, 'good': 1}).values
    test_y3 = test_y[~test_y.isin(['acc', 'vgood'])].replace({'unacc': -1, 'good': 1}).values
    train_X3 = train_X[~train_y.isin(['acc', 'vgood'])]
    test_X3 = test_X[~test_y.isin(['acc', 'vgood'])]
    w3, b3, _ = get_best_model(train_X3, train_y3, test_X3, test_y3)
    print('predict_y3_types', np.unique(np.sign(test_X3 @ w3 + b3), return_counts=True))
    
    # Train a SVM classifier, suppose unacceptable is -1 and very good is 1
    train_y4 = train_y[~train_y.isin(['acc', 'good'])].replace({'unacc': -1, 'vgood': 1}).values
    test_y4 = test_y[~test_y.isin(['acc', 'good'])].replace({'unacc': -1, 'vgood': 1}).values
    train_X4 = train_X[~train_y.isin(['acc', 'good'])]
    test_X4 = test_X[~test_y.isin(['acc', 'good'])]
    w4, b4, _ = get_best_model(train_X4, train_y4, test_X4, test_y4)
    print('predict_y4_types', np.unique(np.sign(test_X4 @ w4 + b4), return_counts=True))
    
    # The final prediction is the majority vote of the four SVM classifiers
    def predict(X):
        arr = np.array([np.sign(X @ w2 + b2), np.sign(X @ w3 + b3), np.sign(X @ w4 + b4)])
        return np.sign(np.sum(arr, axis=0))
    accuracy = np.mean(predict(test_X) == test_y.replace({'unacc': -1, 'acc': 1, 'good': 1, 'vgood': 1}).values)
    print('predict_types', np.unique(predict(test_X), return_counts=True))
    print(f"Vote Accuracy: {accuracy}")

In [12]:
train_general()
train_vote()

c:\Users\11709\anaconda3\Lib\site-packages\cvxpy\reductions\solvers\solving_chain.py:336: FutureWarning: 
    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, specify the ECOS solver explicitly using the ``solver=cp.ECOS`` 
    argument to the ``problem.solve`` method.
    
  warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)


train_gegeral
predict_y1_types (array([-1.]), array([173], dtype=int64))
Accuracy: 0.6705202312138728
train_vote
predict_y2_types (array([-1.]), array([159], dtype=int64))
predict_y3_types (array([-1.]), array([123], dtype=int64))
predict_y4_types (array([-1.,  1.]), array([119,   4], dtype=int64))
predict_types (array([-1.]), array([173], dtype=int64))
Vote Accuracy: 0.6705202312138728


Hi! We plan to train four separate Support Vector Machine (SVM) models with a unique approach.

There's four possible values in the target value: A B C D. 

First Model (Binary Classification for A): In the first model, category A will be labeled as '1', while B, C, and D will be labeled as '-1'. This model's primary objective is to differentiate category A from the others.

Next Three Models (One-vs-One Approach): For the subsequent three models, we will employ a one-vs-one strategy where each model will differentiate A from one of the other categories (B, C, or D). Specifically, the models will be: A=1, B=-1; A=1, C=-1; and A=1, D=-1.

Accuracy Evaluation in Three Phases:

First Phase: we will evaluate the accuracy of the first model (A vs. BCD) independently.

Second Phase: The second evaluation will involve the combined accuracy of the next three models. The prediction for each instance will be the majority vote from these models. If the majority vote is inconclusive, we'll randomly select between the two equally probable classes.

Third Phase: The final accuracy evaluation will combine all four models. The prediction will again be based on the majority vote. In cases where there is a tie (two '1's and two '-1's), we will randomly select between '-1' and '1' for the final prediction.

Do you find this approach feasible and valid for evaluating the performance of SVM models in a multi-class classification problem?


In [13]:
from sklearn import svm
# Split the data into a training set and a test set
#X_train, X_test, y_train, y_test = train_test_split(data[columns], data[6], test_size=0.2, random_state=42)
columns = [0,1,2,3,4,5]

X_train, X_test, y_train, y_test = train_test_split(data[columns], data[6].replace({'unacc': 1, 'acc': 2, 'good': 2, 'vgood': 2}), test_size=0.3)

# Create an instance of the StandardScaler
scaler = MinMaxScaler()

# Normalize the data
train_X = scaler.fit_transform(train_X)
test_X = scaler.transform(test_X)
# Create a SVM classifier with the OvR strategy
clf = svm.SVC(decision_function_shape='ovo')

# Train the classifier
clf.fit(X_train, y_train)

#train accuracy
y_pred = clf.predict(X_train)
print('pred_train_types', np.unique(y_pred, return_counts=True))
accuracy = np.mean(y_pred == y_train)
print(f"Train Accuracy: {accuracy}")

# Make predictions
y_pred = clf.predict(X_test)
print('pred_test_types', np.unique(y_pred, return_counts=True))
# Evaluate the model
accuracy = np.mean(y_pred == y_test)
print(f"Test Accuracy: {accuracy}")

pred_train_types (array([1, 2], dtype=int64), array([877, 332], dtype=int64))
Train Accuracy: 0.9776674937965261
pred_test_types (array([1, 2], dtype=int64), array([365, 154], dtype=int64))
Test Accuracy: 0.9441233140655106
